# Exploring the CCXT Library

Exploring the API functionalities of the CCXT library

In [2]:
import ccxt
import json

binance = ccxt.binance()

## Grabbing all symbols by category

In [26]:
all_symbols = binance.load_markets()

spot_symbols = [
    symbol.replace(':USDT', '').replace('/', '')
    for symbol, details in all_symbols.items()
    if details['quote'] == "USDT" and details['type'] == "spot"
]

futures_symbols = [
    symbol.replace(':USDT', '').replace('/', '')
    for symbol, details in all_symbols.items()
    if details['quote'] == "USDT" and details['info'].get('contractType') == "PERPETUAL" and details['type'] == "future"
]

print("Spot Symbols:", json.dumps(spot_symbols, indent=4))
print("Futures Symbols:", json.dumps(futures_symbols, indent=4))

Spot Symbols: [
    "BTCUSDT",
    "ETHUSDT",
    "BNBUSDT",
    "BCCUSDT",
    "NEOUSDT",
    "LTCUSDT",
    "QTUMUSDT",
    "ADAUSDT",
    "XRPUSDT",
    "EOSUSDT",
    "TUSDUSDT",
    "IOTAUSDT",
    "XLMUSDT",
    "ONTUSDT",
    "TRXUSDT",
    "ETCUSDT",
    "ICXUSDT",
    "VENUSDT",
    "NULSUSDT",
    "VETUSDT",
    "PAXUSDT",
    "BCHABCUSDT",
    "BSVUSDT",
    "USDCUSDT",
    "LINKUSDT",
    "WAVESUSDT",
    "BTTUSDT",
    "USDSUSDT",
    "ONGUSDT",
    "HOTUSDT",
    "ZILUSDT",
    "ZRXUSDT",
    "FETUSDT",
    "BATUSDT",
    "XMRUSDT",
    "ZECUSDT",
    "IOSTUSDT",
    "CELRUSDT",
    "DASHUSDT",
    "NANOUSDT",
    "OMGUSDT",
    "THETAUSDT",
    "ENJUSDT",
    "MITHUSDT",
    "MATICUSDT",
    "ATOMUSDT",
    "TFUELUSDT",
    "ONEUSDT",
    "FTMUSDT",
    "ALGOUSDT",
    "USDSBUSDT",
    "GTOUSDT",
    "ERDUSDT",
    "DOGEUSDT",
    "DUSKUSDT",
    "ANKRUSDT",
    "WINUSDT",
    "COSUSDT",
    "NPXSUSDT",
    "COCOSUSDT",
    "MTLUSDT",
    "TOMOUSDT",
    "PERLUSDT",
    

## Getting Ticker Data

A price ticker contains statistics for a particular market/symbol for some period of time in recent past, usually last 24 hours. The methods for fetching tickers are described below.

```
{
    'symbol':        string symbol of the market ('BTC/USD', 'ETH/BTC', ...)
    'info':        { the original non-modified unparsed reply from exchange API },
    'timestamp':     int (64-bit Unix Timestamp in milliseconds since Epoch 1 Jan 1970)
    'datetime':      ISO8601 datetime string with milliseconds
    'high':          float, // highest price
    'low':           float, // lowest price
    'bid':           float, // current best bid (buy) price
    'bidVolume':     float, // current best bid (buy) amount (may be missing or undefined)
    'ask':           float, // current best ask (sell) price
    'askVolume':     float, // current best ask (sell) amount (may be missing or undefined)
    'vwap':          float, // volume weighed average price
    'open':          float, // opening price
    'close':         float, // price of last trade (closing price for current period)
    'last':          float, // same as `close`, duplicated for convenience
    'previousClose': float, // closing price for the previous period
    'change':        float, // absolute change, `last - open`
    'percentage':    float, // relative change, `(change/open) * 100`
    'average':       float, // average price, `(last + open) / 2`
    'baseVolume':    float, // volume of base currency traded for last 24 hours
    'quoteVolume':   float, // volume of quote currency traded for last 24 hours
}
```

In [22]:

tickers = binance.fetchTickers() # Fetch all tickers
select_tickers = binance.fetchTickers([ 'ETH/BTC', 'BTC/USDT' ]) 

print(json.dumps(select_tickers, indent=4))

{
    "ETH/BTC": {
        "symbol": "ETH/BTC",
        "timestamp": 1704553502109,
        "datetime": "2024-01-06T15:05:02.109Z",
        "high": 0.05148,
        "low": 0.05097,
        "bid": 0.05111,
        "bidVolume": 39.4072,
        "ask": 0.05112,
        "askVolume": 6.7772,
        "vwap": 0.05119017,
        "open": 0.05124,
        "close": 0.05112,
        "last": 0.05112,
        "previousClose": 0.05124,
        "change": -0.00012,
        "percentage": -0.234,
        "average": 0.05118,
        "baseVolume": 25679.97,
        "quoteVolume": 1314.56204306,
        "info": {
            "symbol": "ETHBTC",
            "priceChange": "-0.00012000",
            "priceChangePercent": "-0.234",
            "weightedAvgPrice": "0.05119017",
            "prevClosePrice": "0.05124000",
            "lastPrice": "0.05112000",
            "lastQty": "1.07170000",
            "bidPrice": "0.05111000",
            "bidQty": "39.40720000",
            "askPrice": "0.05112000",
   

## Getting Orderbook Data

Usually there is a separate endpoint for querying current state (stack frame) of the order book for a particular market. An order book is also often called market depth. The order book information is used in the trading decision making process.

To get data on order books, you can use

- fetchOrderBook () // for a single markets order books
- fetchOrderBooks ( symbols ) // for multiple markets order books
- fetchOrderBooks () // for the order books of all markets

In [23]:
orderbook = binance.fetch_order_book('BTC/USD', limit=10)
bid = orderbook['bids'][0][0] if len (orderbook['bids']) > 0 else None
ask = orderbook['asks'][0][0] if len (orderbook['asks']) > 0 else None
spread = (ask - bid) if (bid and ask) else None
print('market price', { 'lowest_bid': bid, 'highest_ask': ask, 'spread': spread })

market price {'lowest_bid': 43822.4, 'highest_ask': 43822.5, 'spread': 0.09999999999854481}


## Public Trades

Can provide info on markets current dynamics.

1. Market Sentiment Analysis:
Analyzing recent trades can help you gauge the overall sentiment in the market. For example, if a series of recent trades are mostly buys, it might suggest bullish sentiment, and vice versa. You can use this information to align your trades with the prevailing sentiment.

2. Confirmation of Price Trends:
Confirming price trends through recent trades can be crucial. If the recent trades align with the trend direction you are considering, it may strengthen your conviction in a trade. Conversely, divergences between recent trade data and the current trend may signal potential reversals.

3. Detection of Large Trades or Whale Activity:
Large trades, often referred to as "whale" activity, can significantly impact market direction. Monitoring recent trades for unusually large transactions can provide insights into potential price movements. This information might be useful for identifying potential support or resistance levels.

4. Identifying Price Reversals or Breakouts:
Patterns in recent trades, such as a sudden increase in trade volume, can be indicative of potential price reversals or breakouts. Traders often look for significant changes in trade activity as a precursor to a change in market direction.

5. Algorithmic Trading Strategies:
Incorporating recent trade data into algorithmic trading strategies can enhance decision-making. For example, you might design a strategy that reacts to specific trade patterns or trade volume changes, automatically executing trades based on predefined criteria.

6. Risk Management:

Monitoring recent trades can also aid in risk management. Unusual trading activity or spikes in volatility may prompt you to reassess your risk exposure and adjust your position sizes accordingly.

In [24]:
trades = binance.fetch_trades("ETHUSDT", limit=10)
print(json.dumps(trades, indent=4))

[
    {
        "info": {
            "a": "987464145",
            "p": "2240.35000000",
            "q": "0.57400000",
            "f": "1271159404",
            "l": "1271159404",
            "T": "1704553500840",
            "m": false,
            "M": true
        },
        "timestamp": 1704553500840,
        "datetime": "2024-01-06T15:05:00.840Z",
        "symbol": "ETH/USDT",
        "id": "987464145",
        "order": null,
        "type": null,
        "side": "buy",
        "takerOrMaker": null,
        "price": 2240.35,
        "amount": 0.574,
        "cost": 1285.9609,
        "fee": null,
        "fees": []
    },
    {
        "info": {
            "a": "987464146",
            "p": "2240.35000000",
            "q": "0.57400000",
            "f": "1271159405",
            "l": "1271159405",
            "T": "1704553501049",
            "m": false,
            "M": true
        },
        "timestamp": 1704553501049,
        "datetime": "2024-01-06T15:05:01.049Z",
      

## Borrow Rates

Margin only. When short trading or trading with leverage on a spot market, currency must be borrowed. Interest is accrued for the borrowed currency.

Data on the borrow rate for a currency can be retrieved using

- fetchCrossBorrowRate () for a single currencies borrow rate
- fetchCrossBorrowRates () for all currencies borrow rates
- fetchIsolatedBorrowRate () for a trading pairs borrow rate
- fetchIsolatedBorrowRates () for all trading pairs borrow rates
- fetchBorrowRatesPerSymbol () for the borrow rates of currencies in individual markets

code (String) Unified CCXT currency code, required (e.g. "USDT")
params (Dictionary) Parameters specific to the exchange API endpoint (e.g. {"settle": "USDT"})

```
{
  currency: 'USDT',  // Unified currency code
  rate: 0.0006,  // A ratio of the rate that interest is accrued at
  period: 86400000,  // The amount of time in milliseconds that is required to accrue the interest amount specified by rate
  timestamp: 1646956800000,  // Timestamp for when the currency had this rate
  datetime: '2022-03-11T00:00:00.000Z',  // Datetime for when the currency had this rate
  info: [ ... ]
}
```

In [25]:
# NOT WORKING

borrow_rates = binance.fetchCrossBorrowRate(code="USDT")
print(json.dumps(borrow_rates, indent=4))

AuthenticationError: binance requires "apiKey" credential

## Funding Rate

Data on the current, most recent, and next funding rates can be obtained using the methods

- fetchFundingRates () for all market symbols
- fetchFundingRates ([ symbol1, symbol2, ... ]) for multiple market symbols
- fetchFundingRate (symbol) for a single market symbol

```
{
    info: { ... },
    symbol: 'BTC/USDT:USDT',
    markPrice: 39294.43,
    indexPrice: 39291.78,
    interestRate: 0.0003,
    estimatedSettlePrice: undefined,
    timestamp: undefined,
    datetime: undefined,
    fundingRate: 0.000072,
    fundingTimestamp: 1645833600000,
    fundingDatetime: '2022-02-26T00:00:00.000Z',
    nextFundingRate: -0.000018,
    nextFundingTimestamp: undefined,
    nextFundingDatetime: undefined,
    previousFundingRate: undefined,
    previousFundingTimestamp: undefined,
    previousFundingDatetime: undefined
}
```

### Funding Rate History Structure

```
{
    info: { ... },
    symbol: "BTC/USDT:USDT",
    fundingRate: -0.000068,
    timestamp: 1642953600000,
    datetime: "2022-01-23T16:00:00.000Z"
}
```

In [ ]:
# Define the trading pair for a perpetual futures contract
symbol = 'BTC/USDT:USDT'

# Fetch the current funding rate
funding_rate = binance.fetch_funding_rate(symbol)
print(json.dumps(funding_rate, indent=4))

# Fetch the funding rate history (Requires API KEY)
# funding_history = binance.fetch_funding_history(symbol)
# print("Funding history:")
# print(json.dumps(funding_history, indent=4))

## Open Interest and Open Interest History 

Open Interest (OI) is a metric used in futures and options trading to measure the total number of outstanding or open contracts at a given time. It represents the total number of contracts that have not been settled (exercised, closed, or expired) by the end of a trading day.

- Increasing Open Interest:
A rise in open interest suggests new money flowing into the market, indicating increased market participation and potential for a sustained price trend.

- Decreasing Open Interest:
A decrease in open interest may signal that market participants are closing positions, potentially indicating a weakening trend or a reversal.

- Stable Open Interest:
Stable open interest may suggest that the current trend is likely to continue, as there is no significant influx or reduction in positions.

- Relationship with Price Movements:
The relationship between open interest and price movements is not always straightforward. It is often used in conjunction with price analysis and other indicators for a more comprehensive view.

- Contrarian Indicator:
Some traders use open interest as a contrarian indicator. For example, if prices are rising while open interest is falling, it could signal potential weakness in the trend.

```
{
    symbol: 'BTC/USDT',
    baseVolume: 80872.801, // deprecated
    quoteVolume: 3508262107.38, // deprecated
    openInterestAmount: 80872.801,
    openInterestValue: 3508262107.38,
    timestamp: 1649379000000,
    datetime: '2022-04-08T00:50:00.000Z',
    info: {
        symbol: 'BTCUSDT',
        sumOpenInterest: '80872.80100000',
        sumOpenInterestValue: '3508262107.38000000',
        timestamp: '1649379000000'
    }
}
```

In [ ]:
open_interest = binance.fetch_open_interest(symbol="BTC/USDT")
open_interest_history = binance.fetch_open_interest_history(symbol="BTC/USDT:USDT")

print(json.dumps(open_interest_history, indent=4))

[
    {
        "symbol": "BTC/USDT:USDT",
        "baseVolume": 69684.424,
        "quoteVolume": 3009186097.027577,
        "openInterestAmount": 69684.424,
        "openInterestValue": 3009186097.027577,
        "timestamp": 1704369300000,
        "datetime": "2024-01-04T11:55:00.000Z",
        "info": {
            "symbol": "BTCUSDT",
            "sumOpenInterest": "69684.42400000",
            "sumOpenInterestValue": "3009186097.02757700",
            "timestamp": "1704369300000"
        }
    },
    {
        "symbol": "BTC/USDT:USDT",
        "baseVolume": 69690.264,
        "quoteVolume": 3009691022.9828033,
        "openInterestAmount": 69690.264,
        "openInterestValue": 3009691022.9828033,
        "timestamp": 1704369600000,
        "datetime": "2024-01-04T12:00:00.000Z",
        "info": {
            "symbol": "BTCUSDT",
            "sumOpenInterest": "69690.26400000",
            "sumOpenInterestValue": "3009691022.98280330",
            "timestamp": "1704369600000"
 

## Historical Volatility


** Can calculate manually
Historical volatility is a statistical measure of the dispersion of returns for a financial instrument over a specific period in the past. It quantifies the degree of variation of a trading instrument's price from its historical average.

```
{
    symbol: 'BTC/USDT',
    baseVolume: 80872.801, // deprecated
    quoteVolume: 3508262107.38, // deprecated
    openInterestAmount: 80872.801,
    openInterestValue: 3508262107.38,
    timestamp: 1649379000000,
    datetime: '2022-04-08T00:50:00.000Z',
    info: {
        symbol: 'BTCUSDT',
        sumOpenInterest: '80872.80100000',
        sumOpenInterestValue: '3508262107.38000000',
        timestamp: '1649379000000'
    }
}
```

Parameters

code (String) required Unified CCXT currency code (e.g. "BTC")
params (Dictionary) Extra parameters specific to the exchange API endpoint (e.g. {"endTime": 1645807945000})

In [ ]:
# DOESNT WORK
# symbol = 'BTC/USDT'
# historical_volatility = binance.historical_volatility(symbol, params={'interval': '1d', 'timeframe': '1w'})
# print(json.dumps(historical_volatility, indent=4))


## Greeks
option only

Use the fetchGreeks method to get the public greeks and implied volatility of an options trading pair from the exchange. The greeks measure how factors like the underlying assets price, time to expiration, volatility, and interest rates, affect the price of an options contract.

In [ ]:
greeks = binance.fetch_greeks("BTC-240126-20000-P", {"category": "options"})
print(json.dumps(greeks, indent=4))

{
    "symbol": "BTC/USDT:USDT-240126-20000-P",
    "timestamp": null,
    "datetime": null,
    "delta": -0.00463603,
    "gamma": 9.7e-07,
    "theta": -4.34842334,
    "vega": 1.43965332,
    "rho": null,
    "bidSize": null,
    "askSize": null,
    "bidImpliedVolatility": 1.29402041,
    "askImpliedVolatility": 1.32873237,
    "markImpliedVolatility": 1.31137639,
    "bidPrice": null,
    "askPrice": null,
    "markPrice": 22.4,
    "lastPrice": null,
    "underlyingPrice": null,
    "info": {
        "symbol": "BTC-240126-20000-P",
        "markPrice": "22.4",
        "bidIV": "1.29402041",
        "askIV": "1.32873237",
        "markIV": "1.31137639",
        "delta": "-0.00463603",
        "theta": "-4.34842334",
        "gamma": "0.00000097",
        "vega": "1.43965332",
        "highPriceLimit": "8716.1",
        "lowPriceLimit": "5"
    }
}
